In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import time
import json
print("Import successful")

Import successful


In [2]:
URI = os.environ["NEO4J_URI"]
USER=os.environ["NEO4J_USER_NAME"]
PASSWORD=os.environ["NEO4J_PASSWD"]
AUTH = (os.environ["NEO4J_USER_NAME"], os.environ["NEO4J_PASSWD"])

In [3]:
#Neo4J connect and Query Boilerplate

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
            
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            #response = (session.run(query, parameters))
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        
        #return pd.DataFrame([r.values() for r in response], columns=response.keys())
        return response
        
    def multi_query(self, multi_line_query, parameters=None, db=None):
        for li in multi_line_query.splitlines():
                print(li)
                result=self.query(li, parameters=None, db=None)
                print(result)

In [4]:
#Make a default connection and it should return `[<Record count(n)=0>]`
conn = Neo4jConnection(uri=URI, 
                       user=USER,
                       pwd=PASSWORD)

#if db is empty, then seed with init values 
res=conn.query('MATCH (n) RETURN count(n)')
print(res)

[<Record count(n)=13589>]


In [ ]:
# NOTE: should be executed only once per database; Alternatively, you can also copy paste this query into neo4j browser
constraint = [
"CREATE CONSTRAINT cooking_id FOR (cooking_method:CookingMethod) REQUIRE cooking_method.itemID IS UNIQUE;",
"CREATE CONSTRAINT cooking_cat_id FOR (cat:CookingMethodCategory) REQUIRE cat.itemID IS UNIQUE;",
"CALL db.awaitIndexes();"]


for cquery in constraint:
    try:
        res = conn.query(cquery)
        print(f"Executed successfully: {cquery}")
    except Exception as e:
        print(f"Error executing query: {cquery}\n{e}")

In [ ]:
# Queries to load nodes
load_cooking_methods = """
CALL apoc.periodic.iterate(
    "CALL apoc.load.csv('/CookingMethods/cooking_methods.csv') YIELD map as row",
    "MERGE (cooking:CookingMethod {itemID: row.id}) 
     ON CREATE SET 
        cooking.name = row.name, 
        cooking.otherNames = row.other_names, 
        cooking.desc = row.description, 
        cooking.DBPediaURL = row.dbpedia_url, 
        cooking.wikiURL = row.wiki_url",
    {batchSize:1000, iterateList:true, parallel:true}
)
"""

load_parent_nodes = """
CALL apoc.periodic.iterate(
    "CALL apoc.load.csv('/CookingMethods/other_nodes.csv') YIELD map as row",
    "MERGE (cooking:CookingMethodCategory {itemID: row.id}) 
     ON CREATE SET 
        cooking.name = row.name",
    {batchSize:1000, iterateList:true, parallel:true}
)
"""

In [7]:
# Execution of those queries
result = conn.query(load_cooking_methods)
print("Cooking Method nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))

result = conn.query(load_parent_nodes)
print("Cooking Method Parent nodes loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Cooking Method nodes loaded. Time taken:0 seconds. Committed Operations: 154. Failed Operations:0
Cooking Method Parent nodes loaded. Time taken:0 seconds. Committed Operations: 34. Failed Operations:0


In [ ]:
# Queries to load relationships
load_relationships = """
CALL apoc.periodic.iterate("CALL apoc.load.csv('/CookingMethods/rel_cooking_methods.csv') yield map as row", 
"MATCH (n {itemID: row.id1})
MATCH (m {itemID: row.id2})
MERGE (n)-[:isChildOf]->(m)", 
{batchSize:10000, iterateList:true, parallel:true})"""

In [9]:
# Execution of these queries
result = conn.query(load_relationships)
print("Cooking Method Relationships loaded. Time taken:" + str(result[0][2]) + ' seconds. Committed Operations: ' + str(result[0][3]) + '. Failed Operations:' + str(result[0][4]))


Cooking Method Relationships loaded. Time taken:3 seconds. Committed Operations: 209. Failed Operations:0


In [ ]:
# Query to view the entire taxonomy
"""
MATCH path = (n:CookingMethod)-[:isChildOf*]->(c:CookingMethodCategory)
RETURN path
"""